<a href="https://colab.research.google.com/github/sokrypton/ColabDesign/blob/v1.0.9/mpnn/example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title install
import os
if not os.path.exists("colabdesign"):
  os.system("pip -q install git+https://github.com/sokrypton/ColabDesign.git@v1.0.9")
  os.system("ln -s /usr/local/lib/python3.7/dist-packages/colabdesign colabdesign")
import sys
import numpy as np
import joblib
import jax
import jax.numpy as jnp
import re
import copy
import random
import haiku as hk
from tqdm import tqdm
from matplotlib import pyplot as plt

from colabdesign.mpnn.wrapper import MPNN_wrapper

from google.colab import files
def get_pdb(pdb_code=""):
  if pdb_code is None or pdb_code == "":
    upload_dict = files.upload()
    pdb_string = upload_dict[list(upload_dict.keys())[0]]
    with open("tmp.pdb","wb") as out: out.write(pdb_string)
    return "tmp.pdb"
  else:
    os.system(f"wget -qnc https://files.rcsb.org/view/{pdb_code}.pdb")
    return f"{pdb_code}.pdb"

# Initialize model

In [2]:
params_path = '/content/colabdesign/mpnn/jax_weights'
model = MPNN_wrapper(params_path=params_path)

Number of edges: 48
Training noise level: 0.02A


In [3]:
# set the provided pdb
inputs = model.prep_inputs(pdb_path=get_pdb("1P3J"),
                           target_chain='A')

## Get the outputs from MPNN

In [5]:
L = len(inputs['dataset_valid'][0]['seq'])
seed = random.randint(0,2147483647)
order = jax.random.normal(jax.random.PRNGKey(seed), (L,))
logits, log_probs = model.score(inputs, order=order)

In [6]:
logits.shape, log_probs.shape

((1, 212, 21), (1, 212, 21))

## Generate sequences

In [7]:
# generate sequences
seqs = model.sampling(inputs, 1, 1)

100%|██████████| 1/1 [00:54<00:00, 54.55s/it]


In [8]:
seqs

['MNLVLGGLPGSGKSTIGELICEDLGVPLISIDDIYRKAIRENTPLGKEALKYIEKGLLVPNELRIGIIEEELSKEECKNGFVLDGYPCTVEEAEALEEILKKLGRPVDLFIYLEKEREVLRERLLSRLVCTECGRSYNLVYRPPKTPGVCDRCGAKLERPFWHKEEVVEIRLEEYEKRVEPLLEYFRKKGKLRKVDAEKDLNEVYEQVKELL']

## Genrate sequences for homomer

In [9]:
inputs = model.prep_inputs(pdb_path=get_pdb("1O91"),
                           target_chain='A B C',
                           ishomomer=True)

In [10]:
seqs = model.sampling(inputs, 1, 1)

100%|██████████| 1/1 [01:34<00:00, 94.32s/it]


In [11]:
seqs

['EKEAFTANLTTPYPPVGEPIKFDKLLYNGQNVYDPNTGIFTCKTPGVYFFSWHGNVYGKDLHVALYKNDEAIQYSYMEYIEGKVSMVSGSAVIELKPGDKVYLECPTEEANGLYAGEDNHSSFSGFLLYET/EKEAFTANLTTPYPPVGEPIKFDKLLYNGQNVYDPNTGIFTCKTPGVYFFSWHGNVYGKDLHVALYKNDEAIQYSYMEYIEGKVSMVSGSAVIELKPGDKVYLECPTEEANGLYAGEDNHSSFSGFLLYET/EKEAFTANLTTPYPPVGEPIKFDKLLYNGQNVYDPNTGIFTCKTPGVYFFSWHGNVYGKDLHVALYKNDEAIQYSYMEYIEGKVSMVSGSAVIELKPGDKVYLECPTEEANGLYAGEDNHSSFSGFLLYET']